**1. Consider the weather.csv data file. This file contains weather data recorded in one minute interval from weather station located in San Diego, California. The weather station is equipped with sensors that capture weather-related measurements such as air temperature, air pressure, and relative humidity. Data was collected for a period of three years, from September 2011 to September 2014, to ensure that sufficient data for different seasons and weather conditions is captured. Each row in weather.csv contains weather data captured for a one-minute interval. In Python, answer the following:**

**(a) (5 points) Using the pandas library, read the csv data file from your S3 bucket and create a data-frame called weather. Select data up to October 31, 2011. After that, remove any observation with missing values.**

**(b) (8 points) After consulting with a meteorologist, he recommends to use the following variables for clustering purposes: air pressure, air temp, avg wind direction, avg wind speed, max wind direction, max wind speed, and relative humidity. Transform all the variables of interest to 0-1 scale.**

**(c) (8 points) Using the silhouette score, estimate the number of clusters for this dataset. Consider 2 to 20 clusters. Make sure to use n init = 20 in the KMeans function from the sklearn.cluster library.**

**(d) (6 points) Using the KMeans function from the sklearn.cluster library, cluster the customers into the number of clusters estimated from part (c).**

**(e) (8 points) Describe each of the clusters. Does the clustering results make sense? if not, suggest how would improve this analysis.**